## Jeffrey Elkner

> **Notes:** 

>> 0. For each test, please provide your null hypothesis H0, alternative hypothesis, HA, test statistic (formula) and the distribution of the test statistic when H0 is true.

>> 0. In each test conclusion, please provide the test statistic value, p-value, and your conclusion against α=0.05.

### Problem 1: Sample means

The same data file used before, *number of named storms* are given in the Excel file: 

(a). Separate the data values into two groups: one for the time period 1970-1994 and the other for 1995-2008.
     Compute means, variances, and standard deviations for the two groups separately. (2 points)


In [1]:
# SOLUTION
import pandas as pd
from ggs560.ggs560_tools import mean
from ggs560.ggs560_tools import variance
from ggs560.ggs560_tools import standard_deviation as sd

file = 'Data/GGS560_HW6_Data.xlsx'
xl = pd.ExcelFile(file)
data = xl.parse('Storm_Data').values

storms1970to1994 = [year[1] for year in data if 1970 <= year[0] <= 1994]
storms1995to2008 = [year[1] for year in data if 1995 <= year[0] <= 2008]

# compute and store means, variances and sample sizes.
size_70to94 = len(storms1970to1994)
size_95to08 = len(storms1995to2008)

mean_70to94 = mean(storms1970to1994)
mean_95to08 = mean(storms1995to2008)

variance_70to94 = variance(storms1970to1994)
variance_95to08 = variance(storms1995to2008)

sd_70to94 = sd(storms1970to1994)
sd_95to08 = sd(storms1995to2008)

def display_data(heading, s, m, v, sd):
    print(heading)
    print("    sample size = {}".format(s))
    print("    mean = {:0.2f}".format(m))
    print("    variance = {:0.2f}".format(v))
    print("    standard deviation = {:0.2f}\n".format(sd))

display_data("Data for Named Storms 1970 to 1994:", size_70to94,
             mean_70to94, variance_70to94, sd_70to94)
          
display_data("Data for Named Storms 1995 to 2008:", size_95to08,
             mean_95to08, variance_95to08, sd_95to08)

Data for Named Storms 1970 to 1994:
    sample size = 25
    mean = 9.28
    variance = 7.13
    standard deviation = 2.67

Data for Named Storms 1995 to 2008:
    sample size = 14
    mean = 14.86
    variance = 21.82
    standard deviation = 4.67



(b). Test if the variances of the two groups are equal. (5 points)

In [2]:
# SOLUTION: Compute the F-stat, degrees of freedom, and critical value
# for alpha = 0.05. If F-stat > critical value, reject the hypothesis
# that the variances are equal.
from scipy.stats import f

F = variance_95to08 / variance_70to94
df = (size_95to08 - 1, size_70to94 - 1)
critval = f.ppf(0.975, df[0], df[1])

msg1 = "The F-stat {:0.2f} is less than the critical value {:0.2f},\n"
msg1 += "so we can not reject that variances are equal."
msg2 = "The F-stat {:0.2f} is greater than the critical value {:0.2f},\n"
msg2 += "so hypothesis of equal variances is rejected."

if critval > F:
    print(msg1.format(F, critval))
else:
    print(msg2.format(F, critval))

The F-stat 3.06 is greater than the critical value 2.50,
so hypothesis of equal variances is rejected.


We can check the above result by hand.

We have $s_1^2 = 21.82$ and $s_2^2 = 7.13$, thus

$$f = \frac{21.82}{7.13} = 3.06$$

With $r_1 = 13$ and $r_2 = 24$, using table A-7 on page 639 of our text, we find that the critical value for $\alpha = 0.05$ is between 2.54 and 2.44 ($r_1 = 12$ and $r_1 = 15$ respectively). $f$ exceeds the critical value, and we reject the null hypothesis.

### Resources:
* https://en.wikipedia.org/wiki/F-test_of_equality_of_variances
* http://www.statisticshowto.com/how-to-conduct-a-statistical-f-test-to-compare-two-variances/
* https://stackoverflow.com/questions/39813470/f-test-with-python-finding-the-critical-value

(c). Based on your conclusions above on variance, choose a right test for testing if the 1995-2008 mean is
     significantly larger than the 1970-1994 mean. (note: if you need to determine the degrees of freedom,
     you may choose a simpler way) (8 points)

In [3]:
# SOLUTION: Since we could not establish that the variances are equal,
# use the unequal variances method. Assume 1995-2008 mean is less than
# or equal to 1970-1994 mean.
from scipy.stats import t

# To make calcuations easier, let's rename some things.
x1 = mean_95to08
x2 = mean_70to94
s1 = variance_95to08
s2 = variance_70to94
n1 = size_95to08
n2 = size_70to94

# With python to do the computation, there is no need to use the
# simpler degrees of freedom estimate.
def df_for_unequal_variances(s1, s2, n1, n2):
    return int((s1/n1 + s2/n2)**2 / ((s1/n1)**2/(n1-1) + (s2/n2)**2/(n2-1)))

df = df_for_unequal_variances(s1, s2, n1, n2)

print("Degrees of freedom = {}.\n".format(df))

# Now use equation 9-4 to find the T-stat
T = (x1 - x2) / (s1 / n1 + s2 / n2) ** 0.5
critval = t.ppf(1-0.05, df)

msg1 = "The T-stat {:0.2f} is less than or equal to the critical\n"
msg1 += "value {:0.2f}, so we can not reject the null hypothesis."
msg2 = "The T-stat {:0.2f} is greater than the critical value {:0.2f}, so the\n"
msg2 += "null hypothesis is rejected, and we conclude with 95% confidence\n"
msg2 += "that the 1995-2008 is significantly greater than the 1970-1994 mean."

if critval >= T:
    print(msg1.format(T, critval))
else:
    print(msg2.format(T, critval))

Degrees of freedom = 17.

The T-stat 4.11 is greater than the critical value 1.74, so the
null hypothesis is rejected, and we conclude with 95% confidence
that the 1995-2008 is significantly greater than the 1970-1994 mean.


### Resources:
* http://sites.nicholas.duke.edu/statsreview/means/welch/

(d). For the 1970 to 1994 data only, test if the mean number of named storms in this particular period is
     significantly different from the long time mean, 8.79, assuming that this long time mean is a fixed
     number. (5 points)

In [4]:
# SOLUTION: Let the null hypothesis be that the 1970 to 1994 mean is
# not significantly different from the population mean of 8.79.
from scipy.stats import t

# Since the standard deviation is not known, use the T-distribution
# with alpha equal to 0.05. The sample size is 25, so df is 24.
T = (mean_70to94 - 8.79) / (sd_70to94 / size_70to94**0.5)
critval = t.ppf(1-0.025, size_70to94-1)

msg1 = "The T-stat {:0.2f} is less than or equal to the critical\n"
msg1 += "value {:0.2f}, so we can not reject the null hypothesis\n"
msg1 += "that the mean of 1970 to 94 storms is significantly\n"
msg1 += "different from the long term mean."
msg2 = "The T-stat {:0.2f} is greater than the critical value {:0.2f}, so the\n"
msg2 += "null hypothesis is rejected, and we conclude with 95% confidence\n"
msg2 += "that the mean for the 1970 to 94 storms is significantly different\n"
msg2 += "from the long term mean."

if critval >= T:
    print(msg1.format(T, critval))
else:
    print(msg2.format(T, critval))

The T-stat 0.92 is less than or equal to the critical
value 2.06, so we can not reject the null hypothesis
that the mean of 1970 to 94 storms is significantly
different from the long term mean.


### Resources:
* https://saylordotorg.github.io/text_introductory-statistics/s12-04-small-sample-tests-for-a-popul.html

### Problem 2: Sample means and paired observations (modified problem 6 of Chapter 9 of the textbook)

Dissolved oxygen data are given for before and after removal of vegetation at same sites along a given river. Please use the data set on the blackboard. The *values are modified* so that they are different from those in the textbook.

(a). Conduct a matched-pair test to determine whether there is a change in dissolved oxygen level following
     weed removal. (4 points)

In [5]:
# SOLUTION: Use problem 6 as a model and rework with given values.
# Null hypotheis: There is no change from before to after, so
# D = 0
data2 = xl.parse('P2_DO')
before = data2['before'].tolist()
after = data2['after'].tolist()
diff = [before[i] - after[i] for i in range(len(before))]

# To help reason about this data, let's display it.
print("BEFORE\tAFTER\tDIFFERENCE")
for i in range(len(before)):
    print("{}\t{}\t{:0.2f}".format(before[i], after[i], diff[i]))
print()
    
n = len(before)
D = mean(diff)
SD = sd(diff)

T = D / (SD / n**0.5)
critval = t.ppf(1-0.025, n-1)

msg1 = "The T-stat {:0.2f} is less than or equal to the critical\n"
msg1 += "value {:0.2f}, so we can not reject the null hypothesis\n"
msg1 += "that there is no significant change in values."
msg2 = "The T-stat {:0.2f} is greater than the critical value {:0.2f}, so the\n"
msg2 += "null hypothesis is rejected, and we conclude with 95% confidence\n"
msg2 += "that there is a significant change in values."

if critval >= T:
    print(msg1.format(T, critval))
else:
    print(msg2.format(T, critval))

BEFORE	AFTER	DIFFERENCE
10.6	10.2	0.40
9.8	9.4	0.40
12.3	11.8	0.50
9.7	9.1	0.60
8.8	8.5	0.30
10.0	8.2	1.80
8.8	8.2	0.60
10.6	10.5	0.10
9.3	9.4	-0.10
9.5	9.1	0.40

The T-stat 3.13 is greater than the critical value 2.26, so the
null hypothesis is rejected, and we conclude with 95% confidence
that there is a significant change in values.


(b). Suppose the hypothesis being tested is that the dissolved oxygen levels have declined. How would this
     change your results? (2 points)

In [6]:
# SOLUTION: We change the null hypothesis to before is
# greater than or equal to after and use a one-tailed
# value for the test
critval = t.ppf(1-0.05, n-1)

msg1 = "The T-stat {:0.2f} is less than or equal to the critical\n"
msg1 += "value {:0.2f}, so we can not reject the null hypothesis\n"
msg1 += "and conclude that there has been a decline in oxygen."
msg2 = "The T-stat {:0.2f} is greater than the critical value {:0.2f}, so we\n"
msg2 += "reject the null hypothesis and conclude with 95% confidence\n"
msg2 += "that oxygen levels have declined."

if critval >= T:
    print(msg1.format(T, critval))
else:
    print(msg2.format(T, critval))

The T-stat 3.13 is greater than the critical value 1.83, so we
reject the null hypothesis and conclude with 95% confidence
that oxygen levels have declined.


(c). If you hadn’t noticed that the data were matched pairs, what might you have concluded in a simple
     difference-of-means test? (4 points) (hints: you may use built-in function to answer the question
     in this part.)

In [7]:
# SOLUTION: We would have to treat this as a difference of
# means test, as in Prob. 1 Part c above.
x1 = mean(before)
x2 = mean(after)
s1 = variance(before)
s2 = variance(after)
n1 = len(before)
n2 = len(after)

df = df_for_unequal_variances(s1, s2, n1, n2)
T = (x1 - x2) / (s1 / n1 + s2 / n2) ** 0.5
critval = t.ppf(1-0.05, df)

msg1 = "The T-stat {:0.2f} is less than or equal to the critical\n"
msg1 += "value {:0.2f}, so we can not reject the null hypothesis\n"
msg1 += "and conclude that there has been a decline in oxygen."
msg2 = "The T-stat {:0.2f} is greater than the critical value {:0.2f}, so we\n"
msg2 += "reject the null hypothesis and conclude with 95% confidence\n"
msg2 += "that oxygen levels have declined."

if critval >= T:
    print(msg1.format(T, critval))
else:
    print(msg2.format(T, critval))

The T-stat 1.03 is less than or equal to the critical
value 1.74, so we can not reject the null hypothesis
and conclude that there has been a decline in oxygen.


### Problem 3 (Bonus Question) Applications

There are two alternative routes between a residential area and a downtown employment center. A transportation planner wants to know whether the mean number of cars using each route during the morning rush hours is the same. A survey over the period of 20 days on both routes reveals that the mean number of cars on Route A is 1040, and the mean number of cars on Route B is 950. Using the survey data, the analyst finds the standard deviations sA=160 and sB=130 cars, respectively. Could you draw a conclusion for the transportation planner? If the same statistics are obtained in 25 survey days, is the conclusion the same? Discuss the reasons if different. Which conclusion is more reliable? (xx points)

> *Hints:* you need to set up the null and alternative hypotheses, define the test statistics, know the null distribution(s), get the p-values and draw the conclusion based on commonly accepted criteria. You may simply the problem by assuming certain conditions.

In [8]:
# SOLUTION: Compute the F-stat, degrees of freedom, and critical value
# for alpha = 0.05. If F-stat > critical value, reject the hypothesis
# that the variances are equal.
from scipy.stats import f

xA = 1040
xB = 950
sA = 160**2
sB = 130**2
nA = 20
nB = 20

F =  xA / xB
df = (nA - 1, nB - 1)
critval = f.ppf(0.975, df[0], df[1])

msg1 = "The F-stat {:0.2f} is less than the critical value {:0.2f},\n"
msg1 += "so we can not reject that variances are equal."
msg2 = "The F-stat {:0.2f} is greater than the critical value {:0.2f},\n"
msg2 += "so hypothesis of equal variances is rejected."

if critval > F:
    print(msg1.format(F, critval))
else:
    print(msg2.format(F, critval))

The F-stat 1.09 is less than the critical value 2.53,
so we can not reject that variances are equal.


In [9]:
# Since we can't reject equal variances, let's assume they are equal.
sP = ((nA - 1) * sA + (nB - 1) * sB) / (nA + nB - 2)
SE = sP * (1/nA + 1/nB)**0.5

# Make the null hypothesis that mean cars in each route are equal,
# so D = 0.
T = (xA - xB) / SE
df = nA + nB - 2
T

0.013393175972477842